Peterson's Solution is a classic algorithm used to solve the critical-section problem in concurrent programming. The critical-section problem arises when multiple processes or threads need to access a shared resource (such as shared memory or a shared data structure) concurrently. To prevent data inconsistency and ensure the correctness of the program, mutual exclusion must be enforced, meaning only one process can access the shared resource at a time.

Peterson's Solution is based on the idea of using shared variables to coordinate the access to the critical section. It is primarily designed for systems with two processes that need to access a shared resource. The algorithm ensures that only one process is in its critical section at any given time, thus providing mutual exclusion.

The algorithm relies on the following shared variables:

- `int turn`: A variable that indicates whose turn it is to enter the critical section.
- `bool flag[2]`: An array of flags that each process uses to indicate its intention to enter the critical section. `flag[i]` is true if process `i` wants to enter the critical section.

The key idea of Peterson's Solution is that before entering the critical section, each process sets its `flag` to true and then sets `turn` to its own process number. Then it enters a loop where it checks if the other process also wants to enter the critical section. If the other process is in the critical section (indicated by its `flag` being true), and it has the turn (indicated by `turn` being equal to the other process number), the current process waits (spins) until it can enter the critical section. If not, it proceeds to enter its critical section.

The pseudocode for Peterson's Solution is as follows:

``` python
# Shared variables
int turn = 0
bool flag[2] = {false, false}

# Process 0
flag[0] = true
turn = 1
while flag[1] and turn == 1:
    # Wait
# Critical section
# Exit section
flag[0] = false

# Process 1
flag[1] = true
turn = 0
while flag[0] and turn == 0:
    # Wait
# Critical section
# Exit section
flag[1] = false
```

The key property of Peterson's Solution is that it provides mutual exclusion, meaning only one process can be in its critical section at a time. Additionally, the algorithm guarantees progress, ensuring that a process that wants to enter the critical section will eventually do so as long as the other process does not remain indefinitely in its critical section.

It's important to note that while Peterson's Solution is a simple and elegant algorithm, it is not suitable for scenarios with more than two processes.

In [ ]:
import threading
import time

cs = 0
flag_0 = False
flag_1 = False
turn = 0

def thread_0():
    global cs, flag_0, flag_1, turn

    flag_0 = True
    turn = 1
    while (flag_1 and turn == 1):
            continue

    for i in range(10):
        cs += 1
        print("Thread 0: cs =", cs)
        time.sleep(0.1)

    flag_0 = False

def thread_1():
    global cs, flag_0, flag_1, turn

    flag_1 = True
    turn = 0
    while (flag_0 and turn == 0):
        continue

    for i in range(10):
        cs += 1000
        print("Thread 1: cs =", cs)
        time.sleep(0.1)

    flag_1 = False

if __name__ == "__main__":
		t0 = threading.Thread(target=thread_0)
		t1 = threading.Thread(target=thread_1)
		t0.start()
		t1.start()

### Alternative

In [ ]:
import threading
import time

# Shared variables
turn = 0
flag = [False, False]

def process_0():
    global turn, flag
    flag[0] = True
    turn = 1
    while flag[1] and turn == 1:
        # Wait
        pass
    # Critical section
    print("Process 0 is in the critical section.")
    time.sleep(2)  # Simulating some work inside the critical section
    # Exit section
    flag[0] = False
    print("Process 0 exited the critical section.\n")

def process_1():
    global turn, flag
    flag[1] = True
    turn = 0
    while flag[0] and turn == 0:
        # Wait
        pass
    # Critical section
    print("Process 1 is in the critical section.")
    time.sleep(1)  # Simulating some work inside the critical section
    # Exit section
    flag[1] = False
    print("Process 1 exited the critical section.\n")

if __name__ == "__main__":
    thread_0 = threading.Thread(target=process_0)
    thread_1 = threading.Thread(target=process_1)

    thread_0.start()
    thread_1.start()

    thread_0.join()
    thread_1.join()

    print("Both processes have completed.")


### Alternative

The producer-consumer problem (or bounded buffer problem) describes two processes, the producer and the consumer, which share a common, fixed-size buffer used as a queue. Producers produce an item and put it into the buffer. If the buffer is already full then the producer will have to wait for an empty block in the buffer. Consumers consume an item from the buffer. If the buffer is already empty then the consumer will have to wait for an item in the buffer. Implement Peterson’s Algorithm for the two processes using shared memory such that there is mutual exclusion between them. The solution should have free from synchronization problems. 

In [ ]:
import random
import time
import threading
import logging
import queue

BSIZE = 8  # Buffer size
PWT = 2  # Producer wait time limit
CWT = 10  # Consumer wait time limit
RT = 10  # Program run-time in seconds

def myrand(n):
    return random.randint(1, n)

def producer(queue, state):
    index = 0
    while state:
        time.sleep(1)
        logging.info("\nProducer is ready now.")
        with queue.lock:
            if not queue.full():
                tempo = myrand(BSIZE * 3)
                logging.info(f"Job {tempo} has been produced")
                queue.put(tempo)
                logging.info(f"Buffer: {list(queue.queue)}")
            else:
                logging.info("Buffer is full, nothing can be produced!!!")
        wait_time = myrand(PWT)
        logging.info(f"Producer will wait for {wait_time} seconds")
        time.sleep(wait_time)

def consumer(queue, state):
    time.sleep(5)
    while state:
        time.sleep(1)
        logging.info("\nConsumer is ready now.")
        with queue.lock:
            if not queue.empty():
                job = queue.get()
                logging.info(f"Job {job} has been consumed")
                logging.info(f"Buffer: {list(queue.queue)}")
            else:
                logging.info("Buffer is empty, nothing can be consumed!!!")
        wait_time = myrand(CWT)
        logging.info(f"Consumer will sleep for {wait_time} seconds")
        time.sleep(wait_time)

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(message)s')

    shared_queue = queue.Queue(BSIZE)
    shared_queue.lock = threading.Lock()
    state = True

    producer_thread = threading.Thread(target=producer, args=(shared_queue, state))
    consumer_thread = threading.Thread(target=consumer, args=(shared_queue, state))

    producer_thread.start()
    consumer_thread.start()

    time.sleep(RT)
    state = False

    producer_thread.join()
    consumer_thread.join()

    logging.info("\nThe clock ran out.")
